# Tools and Routing

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.agents import tool

In [3]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [4]:
search.name

'search'

In [5]:
search.description

'search(query: str) -> str - Search for weather online'

In [6]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [8]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")


In [9]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [10]:
search.args

{'query': {'title': 'Query',
  'description': 'Thing to search for',
  'type': 'string'}}

In [14]:
search.run("fuck you")

'42f'

In [15]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [16]:
get_current_temperature.name

'get_current_temperature'

In [17]:
get_current_temperature.description

'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.'

In [18]:
get_current_temperature.args

{'latitude': {'title': 'Latitude',
  'description': 'Latitude of the location to fetch weather data for',
  'type': 'number'},
 'longitude': {'title': 'Longitude',
  'description': 'Longitude of the location to fetch weather data for',
  'type': 'number'}}

In [19]:
from langchain.tools.render import format_tool_to_openai_function

In [20]:
format_tool_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.',
 'parameters': {'title': 'OpenMeteoInput',
  'type': 'object',
  'properties': {'latitude': {'title': 'Latitude',
    'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'title': 'Longitude',
    'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude']}}

In [24]:
get_current_temperature({"latitude": 1, "longitude": 5})

'The current temperature is 25.8°C'

In [25]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [26]:
search_wikipedia.name

'search_wikipedia'

In [27]:
search_wikipedia.description

'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.'

In [28]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.',
 'parameters': {'title': 'search_wikipediaSchemaSchema',
  'type': 'object',
  'properties': {'query': {'title': 'Query', 'type': 'string'}},
  'required': ['query']}}

In [30]:
search_wikipedia({"query": "what cloudly info limited"})

'Page: .info\nSummary: .info is a generic top-level domain (gTLD) in the Domain Name System (DNS) of the Internet. The name is derived from information, although registration requirements do not prescribe any particular purpose.\nThe TLD info followed ICANN\'s highly publicized announcement, in late 2000, of a phased release of seven new generic top-level domains. The event was the first addition of major gTLDs since the Domain Name System was developed in the 1980s. The seven new gTLDs, selected from over 180 proposals, were meant in part to take the pressure off the com domain.\n\nPage: Oort cloud\nSummary: The Oort cloud (pronounced  AWT or  OORT), sometimes called the Öpik–Oort cloud, is theorized to be a cloud of billions of icy planetesimals surrounding the Sun at distances ranging from 2,000 to 200,000 AU (0.03 to 3.2 light-years). The cloud was proposed in 1950 by the Dutch astronomer Jan Oort, in whose honor the idea was named. Oort proposed that the bodies in this cloud reple

In [31]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [32]:
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [33]:
spec = OpenAPISpec.from_text(text)

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [35]:
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [36]:
pet_openai_functions

[{'name': 'listPets',
  'description': 'List all pets',
  'parameters': {'type': 'object',
   'properties': {'params': {'type': 'object',
     'properties': {'limit': {'type': 'integer',
       'maximum': 100.0,
       'schema_format': 'int32',
       'description': 'How many items to return at one time (max 100)'}},
     'required': []}}}},
 {'name': 'createPets',
  'description': 'Create a pet',
  'parameters': {'type': 'object', 'properties': {}}},
 {'name': 'showPetById',
  'description': 'Info for a specific pet',
  'parameters': {'type': 'object',
   'properties': {'path_params': {'type': 'object',
     'properties': {'petId': {'type': 'string',
       'description': 'The id of the pet to retrieve'}},
     'required': ['petId']}}}}]

In [37]:
from langchain.chat_models import ChatOpenAI

In [38]:
model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

In [39]:
model.invoke("what are three pets names")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'listPets', 'arguments': '{"params":{"limit":3}}'}})

In [40]:
model.invoke("tell me about pet with id 42")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'showPetById', 'arguments': '{"path_params":{"petId":"42"}}'}})

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [41]:
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [43]:
model.invoke("what is the weather in bangladesh right now")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{"latitude":23.685,"longitude":90.3563}'}})

In [44]:
model.invoke("what is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_wikipedia', 'arguments': '{"query":"Langchain"}'}})

In [45]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [47]:
chain.invoke({"input": "do you know bangladesh"})

AIMessage(content='Of course! Bangladesh is a country in South Asia, known for its rich culture, history, and natural beauty. It is bordered by India to the west, north, and east, Myanmar to the southeast, and the Bay of Bengal to the south. Bangladesh is famous for its delicious cuisine, vibrant festivals, and the beautiful Sundarbans mangrove forest, home to the Royal Bengal Tiger. If you have any specific questions about Bangladesh, feel free to ask!')

In [49]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [63]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [65]:
result = chain.invoke({"input": "what is the weather in bd right now"})

In [66]:
type(result)

langchain.schema.agent.AgentActionMessageLog

In [67]:
result.tool

'get_current_temperature'

In [68]:
result.tool_input

{'latitude': 23.685, 'longitude': 90.3563}

In [69]:
get_current_temperature(result.tool_input)

'The current temperature is 33.8°C'

In [70]:
result = chain.invoke({"input": "where you from!"})

In [71]:
type(result)

langchain.schema.agent.AgentFinish

In [72]:
result.return_values

{'output': "I exist in the digital realm, so I don't have a physical location like you do. But I'm here to help you with any questions or tasks you have!"}

In [73]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [74]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [75]:
result = chain.invoke({"input": "What is the weather in bd francisco right now?"})

In [76]:
result

'The current temperature is 17.0°C'

In [79]:
result = chain.invoke({"input": "What is langraph?"})

In [80]:
result

'Page: List of After War Gundam X episodes\nSummary: After War Gundam X is a 1996 Japanese mecha drama anime series and the seventh incarnation to Sunrise\'s long-running Gundam franchise. It is directed by Shinji Takamatsu and written by Hiroyuki Kawasaki. It aired on TV Asahi from April 5, to December 28, 1996. From episodes 1–26, the first opening theme is "DREAMS" by Romantic Mode while the ending theme from episodes 1–13 is "HUMAN TOUCH" by Warren Wiebe and the second ending theme from episodes 14–26 is a Japanese version of "HUMAN TOUCH" by re-kiss. From episodes 27–39, the second opening theme is "Resolution" by Romantic Mode while the third ending theme up to episode 38 is "Gin\'iro Horizon" (銀色Horizon, Silver Horizon) by Satomi Nakase. For episode 39, "HUMAN TOUCH" is the ending theme. Episode titles are taken from quotes spoken by characters in the series.'

In [81]:
chain.invoke({"input": "hi!"})

'Hello! How can I assist you today?'